In [29]:
import cv2
import numpy as np

# Load pre-trained model
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

# Load COCO class names (used for YOLO)
classes = []
with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")

# Get the output layer names
layer_names = net.getUnconnectedOutLayersNames()

# Open video
cap = cv2.VideoCapture('example.mp4')

# List to store tracked people centroids
tracked_centroids = []

# Function to calculate Intersection over Union (IOU) between two bounding boxes
def calculate_IOU(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    intersect_x1 = max(x1, x2)
    intersect_y1 = max(y1, y2)
    intersect_x2 = min(x1 + w1, x2 + w2)
    intersect_y2 = min(y1 + h1, y2 + h2)

    if intersect_x2 > intersect_x1 and intersect_y2 > intersect_y1:
        intersect_area = (intersect_x2 - intersect_x1) * (intersect_y2 - intersect_y1)
        box1_area = w1 * h1
        box2_area = w2 * h2

        iou = intersect_area / float(box1_area + box2_area - intersect_area)
        return iou

    return 0.0

# YOLO detection threshold and frame skip
detection_threshold = 0.5
frame_skip = 30

max_people_count = 0  # Variable to keep track of the highest people count

while cap.isOpened():
    for _ in range(frame_skip - 1):
        cap.read()
    ret, frame = cap.read()

    if ret:
        # Prepare frame for object detection (resize, normalization, etc.)
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)

        # Perform object detection
        detections = net.forward(layer_names)

        new_centroids = []

        for detection in detections:
            for obj in detection:
                scores = obj[5:]
                class_id = scores.argmax()
                confidence = scores[class_id]

                if classes[class_id] == 'person' and confidence > detection_threshold:
                    bbox = obj[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                    bbox = tuple(bbox.astype(int))

                    centroid = (bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2

                    new_centroids.append(centroid)

        # Update tracked centroids
        if tracked_centroids:
            updated_centroids = []
            for centroid in new_centroids:
                is_tracked = False
                for tracked_centroid in tracked_centroids:
                    if calculate_IOU((centroid[0], centroid[1], 1, 1), (tracked_centroid[0], tracked_centroid[1], 1, 1)) > 0.5:
                        updated_centroids.append(tracked_centroid)
                        is_tracked = True
                        break
                if not is_tracked:
                    updated_centroids.append(centroid)
            tracked_centroids = updated_centroids
        else:
            tracked_centroids = new_centroids

        # Display tracked objects
        for centroid in tracked_centroids:
            x, y = int(centroid[0]), int(centroid[1])
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        
        num_people = len(tracked_centroids)
        if num_people > max_people_count:
            max_people_count = num_people  # Update the maximum count
        
        cv2.putText(frame, f'People: {num_people}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('VIEW', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f'Highest people count in a frame: {max_people_count}')
